# Polychromatic PSF simulation with Scoob
The scoobPSF package builds a layer on-top of poppy that is specific to the scoob optical bench.
This notebook demonstrates how to simulate polychromatic PSFs with the scoobPSF package. <br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import astropy.units as u
from astropy.io import fits
from pathlib import Path
from IPython.display import clear_output, display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from importlib import reload

import poppy
import ray

import logging, sys
poppy_log = logging.getLogger('poppy')
poppy_log.setLevel('DEBUG')
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
poppy_log.disabled = True

import scoobpsf
from scoobpsf.math_module import xp, _scipy
from scoobpsf.imshows import *
from scoobpsf import scoobm

## Optical System Parameters
The following cell provides parameters of the optical system

In [ ]:
pupil_diam = 6.75*u.mm 
lyot_diam=3.6*u.mm
det_rotation = 20 #  degrees of rotation of the detector relative to the optical axis

## Simulation Parameters
The following cell provides parameters which are specific to the simulation

In [ ]:
wavelength_c = 632.8e-9*u.m # central wavelength
npix=512/4
oversample=16/4
use_opds=False #  Incorporate WFE from each optical component

### Set wavelength sampling parameters
Note that the bandwidth is to be set in nm, but is provided below as a percentage of the central wavelength.
The parallelization of the simulation follows from the number of wavelengths (nlam.

In [ ]:
bandwidth=0.1*wavelength_c # Fractional bandwidth
bandwidth=10 * u.nm
nlam = 3 # number of wavelengths, which is also the number of actors

In [ ]:
minlam = wavelength_c - bandwidth/2
maxlam = wavelength_c + bandwidth/2
wavelengths = np.linspace( minlam, maxlam, nlam )
print(f'PSF will be built using wavelengths: {wavelengths}')

### Set source flux for each wavelength
Array will be normalized so the integrated flux is 1.

In [ ]:
## Set Source Flux for each wavelength
f_lambda=xp.ones(len(wavelengths))
f_lambda/=xp.sum(f_lambda)
print(f_lambda)

## Declare the coronograph setup
Note that by default, the coronograph is not installed when the class gets instantiated.

In [ ]:
vortex = scoobpsf.agpm.IdealAGPM(name='VVC', wavelength=wavelength_c, charge=6, singularity=20*u.um)
lyot_stop = poppy.CircularAperture(name='Lyot Stop', radius=lyot_diam/2.0)

#### Optional: Use the knife edge as a FPM
Knife edge shifted to 2 lambda/D.<br>
Note that he f/# at the FPM is 48.

In [ ]:
shift = (2 * wavelength_c * 48).to("um")
knife_edge = poppy.KnifeEdge(name='Knife Edge', rotation=0, shift_x=-shift)

## Initialize the classes with the appropriate parameters for a non-coronographic PSF

In [ ]:
scoob_kwargs = {'npix':npix, 
          'oversample':oversample, 
          'det_rotation':det_rotation, 
          'use_opds':use_opds,
          'pupil_diam':pupil_diam,
}

Instantiate the scoobm model class, which is used to calculate several key numbers

In [ ]:
reload(scoobm)
model = scoobm.SCOOBM(**scoob_kwargs)

Instantiate a ray actor class from the original scoobm class 

In [ ]:
ray_scoobm = ray.remote(scoobm.SCOOBM)

Create one actor per wavelength

In [ ]:
actors = []
for i in range(nlam):
    actors.append(ray_scoobm.options(num_gpus=1/nlam).remote(**scoob_kwargs))
    # Set actor specific keywords
    actors[i].setattr.remote('wavelength', wavelengths[i])

Instantiate the parallelizedScoob class which handles the actors

In [ ]:
reload(scoobm)
from scoobpsf.scoobm import ParallelizedScoob
p_scoob=ParallelizedScoob(actors,f_lambda)

## Add desired shape on the DM

In [ ]:
# The flattest wavefront obtained for the system in the lab by
# performing phase diversity
dm_flat = fits.getdata(scoobm.module_path/'scoob_dm_flat.fits')

In [ ]:
p_scoob.set_dm(dm_flat)

## Run all wavelengths unocculted to determine normalization factors
Setting the actor attributes to None first ensures the correct masks are in which is helpful if cells are not run sequentially.

In [ ]:
p_scoob.set_actor_attr('FPM',None)
p_scoob.set_actor_attr('LYOT',None)

psfs_unocc = p_scoob.snaps()
imshow1(psfs_unocc[0], f'Image for wavelength {wavelengths[0].to("nm"):0.1f}', lognorm=True)

### First insert FPM & Lyot Mask to determine the occulted PSFs

In [ ]:
p_scoob.set_actor_attr('FPM',vortex)
p_scoob.set_actor_attr('LYOT',lyot_stop)

In [ ]:
# DO NOT NORMALIZE - this will be done later
### Set normalization factor based on the maximum of the unocculted value
# for i in range(nlam):
#     actors[i].setattr.remote('im_norm', )

Create array of unocculted and non-normalized PSFs

In [ ]:
psfs_occ = p_scoob.snaps()

Display a slice

In [ ]:
i=0
norm=psfs_unocc[i].max()
imshow1(psfs_occ[i]/norm, f'Normalized Image for wavelength {wavelengths[0].to("nm"):0.1f}', lognorm=True)

## Perform intensity weighting and combination of PSFs
This section creates unocculted and occulted PSFs based on the desired weighting at each wavelength. <br>
The spectrum is normalized such that the integral over the wavelength range will be 1.

In [ ]:
# This is just for debugging convenience
reload(scoobm)
from scoobpsf.scoobm import ParallelizedScoob
p_scoob=ParallelizedScoob(actors,f_lambda)

In [ ]:
psf_occ_calib, psf_unocc_calib = ParallelizedScoob.flux_calibrate_psf(psfs_occ, f_lambda, norm=psfs_unocc)

Need pixel scale for display. <br>
The pixelscale of 4.63um corresponds to ~0.2 lam/D per pixel using the 150mm final imaging lens. <br>
The value was determined empirically by injecting sinusoidal waves. <br>
FIXME: this is hard coded in the model but should come up a few levels to the instrument configuration level

In [ ]:
psf_pixelscale = 4.63e-6*u.m/u.pix
psf_pixelscale_lamD = (1/(5)) * psf_pixelscale.to(u.m/u.pix).value/4.63e-6

In [ ]:
imshow1(psf_occ_calib, lognorm=True, pxscl=model.psf_pixelscale_lamD)

## Calculation of Wavefronts at each optic/plane
The calc_wfs method can be used to calculate the wavefront at each plane.<br>
Note that these are not normalized by the un-occulted PSF amplitudes.

This is not yet implemented here. This is better accomplished using the monochromatic demo.